# Chatbot: Load data

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
BASE_PATH             = '../../..'
LIB_PATH              = f'{BASE_PATH}/lib'
DATASET_PATH          = f'{BASE_PATH}/datasets'

In [4]:
import sys
sys.path.append(LIB_PATH)

import numpy as np
import pandas as pd

import util as ut

import torch
import data as dt
import data.dataset as ds

2023-12-31 17:09:38.676419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 17:09:39.139802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-31 17:09:39.588093: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-31 17:09:39.601508: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

<Figure size 640x480 with 0 Axes>

In [5]:
def to_tensor(obs, device, columns): 
    data = obs[columns]
    if type(data) == pd.DataFrame:
        data = data.values
    return torch.tensor(data).to(device)

features_fn = lambda obs, device: to_tensor(obs, device, ['user_seq', 'movie_seq'])
target_fn   = lambda obs, device: to_tensor(obs, device, ['user_movie_rating'])

In [6]:
dataset = ds.MovieLensTMDBDatasetFactory.from_path(
    path             = DATASET_PATH,
    transform        = features_fn,
    target_transform = target_fn,
    device           = torch.device('cpu'),
    filter_fn        = lambda df: df[(df['user_movie_rating_year'] >= 2004)]
)

In [53]:
items = dataset \
    .data[['movie_id', 'movie_title', 'movie_genres', 'movie_overview', 'movie_release_year', 'movie_imdb_id']] \
    .drop_duplicates(subset=['movie_id'])

In [49]:
import requests
import json

In [51]:
base_url = 'http://nonosoft.ddns.net:8080/api/v1'

error_items = []
page        = []
page_counter = 0
for _, row in items.iterrows():
    if page_counter < 10:
        page.append({
            'id'          : str(row['movie_id']),
            'title'       : row['movie_title'],
            'description' : row['movie_overview'],
            'genres'      : row['movie_genres'],
            'release'     : str(row['movie_release_year']),
            'imdb_id'     : str(row['movie_imdb_id'])
        })
        page_counter += 1
    else:
        try:
            headers =  {"Content-Type":"application/json"}
            response = requests.post(f'{base_url}/items/bulk', data=json.dumps(page), headers=headers)
        except Exception as e:
            print(e)
            error_items.extend(page)
        finally:
            page_counter = 0
            page         = []